# 객관식 문제 변형(251111)

In [2]:
import os, json
from tools.pipeline.config import ONEDRIVE_PATH

with open('multiple_classify.json', 'r', encoding='utf-8') as f:
    multiple = json.load(f)

## 모두고르시오

### 변형하기

In [ ]:
from tools.core.llm_query import LLMQuery
llm = LLMQuery()

system_prompt = """당신은 프롬프트 전문가입니다. 다음은 ㄱ,ㄴ,ㄷ,ㄹ / 가,나,다,라 등 보기 중 옳은 것을 모두 고르는 문제입니다. 다음 문제의 보기를 찾아 선택지로 바꾸고 (예: ① ㄱ, ② ㄴ, ③ ㄷ, ④ ㄹ / ① 가, ② 나, ③ 다, ④ 라)
주어진 문제와 정답, 해설을 참고하여 선택지 중 옳은 것을 모두 고르는 문제로 변형하려고 합니다. 아래와 같은 프롬프트를 사용하려고 할 때, 올바른 프롬프트인지 확인해주세요."""
user_prompt = """"========================= 아래 ==========================
당신은 25년 경력의 문제 출제 전문가입니다.

변형 규칙
1) '모두 고르시오' 문제의 대상(보기)를 선택지로 바꿉니다(예: ① ㄱ, ② ㄴ, ③ ㄷ, ④ ㄹ / ① 가, ② 나, ③ 다, ④ 라).
2) 기존의 정답을 참고하여 옳은 선택지를 모두 고릅니다. (예: 기존 정답이 ㄱ,ㄷ 이면 새로운 정답은 ①,③).
3)그 외 문장과 LaTeX 수식이나 테이블 표현은 원형을 최대한 보존합니다.

  출력 형식
  {{
    "question_id": "문제번호",
    "question": "문제",
    "options": ["① 선택지1", "② 선택지2", "③ 선택지3", "④ 선택지4", "⑤ 선택지5"],
    "answer": ["정답번호1", "정답번호2", ...], 
    "explanation": "① 옳다(원래 답 중 하나): 근거 ... / ③ 옳다(원래 답 중 하나): 근거 ... / ⑤ 옳지 않다: 근거 ..."
  }}"""

response = llm.query_openrouter(system_prompt, user_prompt, model_name='openai/gpt-5')

ModuleNotFoundError: No module named 'transformers'

In [ ]:
from tools.core.llm_query import LLMQuery
import time
import logging
from datetime import datetime

# 로깅 설정
log_dir = 'logs'
os.makedirs(log_dir, exist_ok=True)
log_file = os.path.join(log_dir, f'multiple_answer_abcd.log')

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(log_file, encoding='utf-8'),
        logging.StreamHandler()
    ]
)

logger = logging.getLogger(__name__)

llm = LLMQuery()
output_dir = os.path.join('7_multiple_rw/pick_right')

logger.info(f"로그 파일: {log_file}")
logger.info("=" * 80)

for i in range(2, 6):
  logger.info(f"\n[정답 {i}개 그룹] 처리 시작")
  parsed_responses = []
  not_parsed_responses = []
  no_responses = []

  for idx, p in enumerate(pick_abcd_sampling, 1):
    question_id = p['file_id'] + '_' + p['tag']
    logger.info(f"\n[{i}개 그룹] {idx}/{len(pick_abcd_sampling)} - 문제 ID: {question_id} 처리 중...")
    options_ct = len(p['options'])

    if options_ct - i == 1:
      # 옳은 답이 1개여야 하는 경우
      system_prompt = f"""당신은 25년 경력의 문제 출제 전문가입니다.

  변형 규칙
  1) 주어진 답(원래의 ‘옳은’ 선택지)은 그대로 유지합니다.
  3) 문제 지문을 ‘옳지 않은 것을 모두 고르시오’로 명확히 바꿉니다. 그 외 문장과 LaTeX 수식이나 테이블 표현은 원형을 최대한 보존합니다.
  4) 새로운 정답은 변형 후 ‘옳지 않은’ 선택지 전부입니다(총 선택지 수가 {options_ct}개라면 {i}개).

  출력 형식
  {{
    "question_id": "문제번호",
    "question": "변형된 문제(옳지 않은 것을 모두 고르시오)",
    "options": ["① 선택지1", "② 선택지2", "③ 선택지3", "④ 선택지4", "⑤ 선택지5"],
    "answer": ["정답번호1", "정답번호2", ...], 
    "explanation": "① 옳지 않다: 근거 ... / ③ 옳다(원래 답): 근거 ... / ⑤ 옳지 않다: 근거 ..."
  }}"""
      
    elif options_ct - i == 0:
      # 옳은 답이 0개여야 하는 경우
      system_prompt = f"""당신은 25년 경력의 문제 출제 전문가입니다.

  변형 규칙
  1) 주어진 답(원래의 ‘옳은’ 선택지)을 단어 1~2개 수준의 최소 변경으로 ‘옳지 않은’ 선택지로 만듭니다. (ex. 높다 -> 낮다, 한다 -> 하지 않는다)
  2) 문제 지문을 ‘옳지 않은 것을 모두 고르시오’로 명확히 바꿉니다. 그 외 문장과 LaTeX 수식이나 테이블 표현 원형을 최대한 보존합니다.
  3) 새로운 정답은 변형 후 ‘옳지 않은’ 선택지 전부입니다(총 선택지 수가 {options_ct}개라면 {i}개).
  4) 해설에는 각 선택지의 옳고 그름과 간단한 근거를 명시하세요. 특히 새로 만든 정답 1개의 변경 포인트를 밝혀주세요.

  출력 형식
  {{
    "question_id": "문제번호",
    "question": "변형된 문제(옳지 않은 것을 모두 고르시오)",
    "options": ["① 선택지1", "② 선택지2", "③ 선택지3", "④ 선택지4", "⑤ 선택지5"],
    "answer": ["정답번호1", "정답번호2", ...], 
    "explanation": "① 옳지 않다: 근거 ... / ③ 옳지 않다: 근거 ... / ⑤ 옳지 않다(원래 답): 변경 단어 ‘높다→낮다’, 근거 ..."
  }}

  비고
  - 원문 선택지 수가 5개가 아닐 경우에도 동일 원칙을 적용합니다(총 {options_ct-i-1}개 추가 오답, 나머지 전부 정답).
  - 사실과 상충하는 임의 창작을 피하고, 제공된 해설의 논리 범위 내에서만 최소 변경을 수행하세요."""
          
    else:
      # 옳은 답이 2개 이상인 경우
      system_prompt = f"""당신은 25년 경력의 문제 출제 전문가입니다.

  변형 규칙
  1) 주어진 답(원래의 ‘옳은’ 선택지)은 그대로 유지합니다.
  2) 나머지 선택지 중에서 정확히 {options_ct-i-1}개를 골라, 단어 1~2개 수준의 최소 변경으로 ‘옳지 않은’ 선택지로 만듭니다. (ex. 높다 -> 낮다, 한다 -> 하지 않는다)
  3) 문제 지문을 ‘옳지 않은 것을 모두 고르시오’로 명확히 바꿉니다. 그 외 문장과 LaTeX 수식이나 테이블 표현 원형을 최대한 보존합니다.
  4) 새로운 정답은 변형 후 ‘옳지 않은’ 선택지 전부입니다(총 선택지 수가 {options_ct}개라면 {i}개).
  5) 해설에는 각 선택지의 옳고 그름과 간단한 근거를 명시하세요. 특히 새로 만든 오답 1개의 변경 포인트를 밝혀주세요.

  출력 형식
  {{
    "question_id": "문제번호",
    "question": "변형된 문제(옳은 것을 모두 고르시오)",
    "options": ["① 선택지1", "② 선택지2", "③ 선택지3", "④ 선택지4", "⑤ 선택지5"],
    "answer": ["정답번호1", "정답번호2", ...], 
    "explanation": "① 옳지 않다: 근거 ... / ③ 옳다(원래 답): 근거 ... / ⑤ 옳다(변형): 변경 단어 ‘높다→낮다’, 근거 ..."
  }}

  비고
  - 원문 선택지 수가 5개가 아닐 경우에도 동일 원칙을 적용합니다(총 {options_ct-i-1}개 추가 오답, 나머지 전부 정답).
  - 사실과 상충하는 임의 창작을 피하고, 제공된 해설의 논리 범위 내에서만 최소 변경을 수행하세요."""
    
    user_prompt = f"""
========== 다음 ===========
"""
    question_id = p['file_id'] + '_' + p['tag']
    question = p['question']
    options = p['options']
    answer = p['answer']
    explanation = p['explanation']

    user_prompt += f"""문제번호: {question_id}
  문제: {question}
  선택지: {options}
  답: {answer}
  해설: {explanation}
  """
    logger.info(f"  - API 호출 중... (모델: openai/o3)")
    time.sleep(0.6)
    try:
      response = llm.query_openrouter(system_prompt, user_prompt, model_name='openai/o3')
      logger.info(f"  - API 응답 수신 완료 (길이: {len(response) if response else 0}자)")
      try:
        # 단일 JSON 객체 파싱 시도
        parsed_response = None
        # 먼저 배열 형태로 시도
        parsed_response = llm.parse_api_response(response)
        
        # 배열이 아니면 단일 객체로 시도
        if parsed_response is None:
          try:
            # JSON 객체 찾기
            start_idx = response.find('{')
            end_idx = response.rfind('}') + 1
            
            if start_idx != -1 and end_idx > 0:
              json_str = response[start_idx:end_idx]
              parsed_response = json.loads(json_str)
              logger.info(f"  - 단일 JSON 객체 파싱 성공")
          except Exception as parse_err:
            logger.warning(f"  - JSON 객체 파싱 실패: {parse_err}")
            pass
        
        if parsed_response is not None:
          parsed_responses.append(parsed_response)
          logger.info(f"  ✓ 파싱 성공 - result.json에 저장")

          # 기존 파일이 있으면 읽어서 누적
          result_file = f'{output_dir}/{i}/result.json'
          os.makedirs(os.path.dirname(result_file), exist_ok=True)
          
          existing_results = []
          if os.path.exists(result_file):
            try:
              with open(result_file, 'r', encoding='utf-8') as f:
                existing_data = json.load(f)
                # 기존 데이터가 리스트면 그대로 사용, 아니면 리스트로 변환
                if isinstance(existing_data, list):
                  existing_results = existing_data
                else:
                  existing_results = [existing_data]
            except:
              existing_results = []
          
          # 새 결과 추가
          existing_results.append(parsed_response)
          
          # 누적된 결과 저장
          with open(result_file, 'w', encoding='utf-8') as f:
            json.dump(existing_results, f, ensure_ascii=False, indent=4)
          logger.info(f"  ✓ 저장 완료 (총 {len(existing_results)}개 결과 누적)")
        else:
          # 파싱 실패
          not_parsed_responses.append((p, response))
          logger.warning(f"  ✗ 파싱 실패 - not_parsed.json에 저장")

          # 기존 파일이 있으면 읽어서 누적
          not_parsed_file = f'{output_dir}/{i}/not_parsed.json'
          os.makedirs(os.path.dirname(not_parsed_file), exist_ok=True)
          
          existing_not_parsed = []
          if os.path.exists(not_parsed_file):
            try:
              with open(not_parsed_file, 'r', encoding='utf-8') as f:
                existing_not_parsed = json.load(f)
            except:
              existing_not_parsed = []
          
          # 새 결과 추가
          existing_not_parsed.append((p, response))
          
          # 누적된 결과 저장
          with open(not_parsed_file, 'w', encoding='utf-8') as f:
            json.dump(existing_not_parsed, f, ensure_ascii=False, indent=4)
          logger.warning(f"  ✓ 저장 완료 (총 {len(existing_not_parsed)}개 파싱 실패 누적)")

      except Exception as e:
        # 파싱 실패
        not_parsed_responses.append((p, response))
        logger.error(f"  ✗ 파싱 중 예외 발생: {str(e)}")

        # 기존 파일이 있으면 읽어서 누적
        not_parsed_file = f'{output_dir}/{i}/not_parsed.json'
        os.makedirs(os.path.dirname(not_parsed_file), exist_ok=True)
        
        existing_not_parsed = []
        if os.path.exists(not_parsed_file):
          try:
            with open(not_parsed_file, 'r', encoding='utf-8') as f:
              existing_not_parsed = json.load(f)
          except:
            existing_not_parsed = []
        
        # 새 결과 추가
        existing_not_parsed.append((p, response))
        
        # 누적된 결과 저장
        with open(not_parsed_file, 'w', encoding='utf-8') as f:
          json.dump(existing_not_parsed, f, ensure_ascii=False, indent=4)
        logger.warning(f"  ✓ 저장 완료 (총 {len(existing_not_parsed)}개 파싱 실패 누적)")

    except Exception as e:
      # 응답 없음
      response = None
      no_responses.append(p)
      logger.error(f"  ✗ API 호출 실패: {str(e)}")

      # 기존 파일이 있으면 읽어서 누적
      no_response_file = f'{output_dir}/{i}/no_response.json'
      os.makedirs(os.path.dirname(no_response_file), exist_ok=True)
      
      existing_no_responses = []
      if os.path.exists(no_response_file):
        try:
          with open(no_response_file, 'r', encoding='utf-8') as f:
            existing_no_responses = json.load(f)
        except:
          existing_no_responses = []
      
      # 새 결과 추가
      existing_no_responses.append(p)
      
      # 누적된 결과 저장
      with open(no_response_file, 'w', encoding='utf-8') as f:
        json.dump(existing_no_responses, f, ensure_ascii=False, indent=4)
      logger.warning(f"  ✓ 저장 완료 (총 {len(existing_no_responses)}개 응답 없음 누적)")
    
    logger.info(f"  [{i}개 그룹] {idx}/{len(pick_right_sampling[i])} - 문제 ID: {question_id} 처리 완료")
    # break
  
  logger.info(f"\n[정답 {i}개 그룹] 처리 완료 - 성공: {len(parsed_responses)}, 파싱실패: {len(not_parsed_responses)}, 응답없음: {len(no_responses)}")
  logger.info("=" * 80)
  # break

In [ ]:
# 5문제: 1분 10초